<a href="https://colab.research.google.com/github/ivonnics/Machine-Learning/blob/master/Demo_4_Differences_between_two_(2)_CSV_files_in_my_Google_Drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
pd.__version__

## Mount the Google Drive to Google Colab

In [0]:
# https://towardsdatascience.com/downloading-datasets-into-google-drive-via-google-colab-bcb1b30b0166
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [0]:
import sys, sqlite3, pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
files = ['/content/drive/My Drive/Lista.csv', 
         '/content/drive/My Drive/Listita.csv']
df2=pd.read_csv('/content/drive/My Drive/Listita.csv')
df1=pd.read_csv('/content/drive/My Drive/Lista.csv')
#df1 = pd.read_csv(files[0])
#df2 = pd.read_csv(files[1])
print(df1)
print('________________________')
print(df2)
df3 = (df1 != df2).any(axis=None)
print('Differences in file:', df3)
df3 = (df1 != df2).any(1)
ne_stacked = (df1 != df2).stack()
changed = ne_stacked[ne_stacked]
changed.index.names = ['id', 'col']
print('Differences In:')
print(changed)

In [0]:
df3 = df1.eq(df2)
print(df3.all())
print('-----------')
print(df3.all(axis=1))
print('____________')
df4 = df3.all(axis=1)
df4 = pd.DataFrame(df4, columns=['Columns'])
print(df4[df4['Columns']==False])

In [0]:
#This is the correct mismtaching column


conn = sqlite3.connect(':memory:') #we are spinning an SQL db in memory
cur = conn.cursor()
chunksize = 10000
i=0
for file in files:
    i = i+1
    for chunk in pd.read_csv(file, chunksize=chunksize): #load the file in chunks in case its too big
        chunk.columns = chunk.columns.str.replace(' ', '_') #replacing spaces with underscores for column names
        chunk.to_sql(name='file' + str(i), con=conn, if_exists='append')
print('Comparing', files[0], 'to', files[1]) #Compare if all data from File[0] are present in File[1]
cur.execute( '''SELECT * FROM File1
                EXCEPT
                SELECT * FROM File2''')
i=0
for row in cur:
    print(row)
    i=i+1
if i==0: print('No Differences')
print('Comparing', files[1], 'to', files[0]) #Compare if all data from File[1] are present in File[0]
cur.execute( '''SELECT * FROM File2
                EXCEPT
                SELECT * FROM File1''')
i=0
for row in cur:
    print(row)
    i=i+1
if i==0: print('No Differences')
cur.close()